In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

First, we load the required libraries:

In [2]:
# Set path for fastai directory
import sys
sys.path.append("/Users/kunal/Desktop/MSAN_Coursework/621_ML/fastai")

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [97]:
from functools import reduce

In [4]:
PATH = "~/data/grocery_sales"

## What Does Our Data Look Like?
Let's load the data from the files provided for the competition and take a look at it.

### Training Data
The training data, `train.csv` is a large file (~5 GB).  On a machine with relatively low RAM (e.g. 8 GB), attempting to load the entire file in a pandas DataFrame can lead to failure caused by running out of memory.  One way of fixing this issue is to make use of the `low_memory=True` argument of `read_csv`.  With this method, the csv file is processed in chunks requiring lower memory usage, while at the same time reading the csv's contents into a single DataFrame.

However, the dtypes of the columns of the DataFrame must be specified in `read_csv` if we wish to set `low_memory=True`.  This is because not specifying dtypes forces pandas to guess column dtypes - which is a memory-intensive task.  Please see this Stack Overflow answer for a additional explanation:
https://stackoverflow.com/a/27232309


We first create a new file called `small_train.csv` using only the first row of data from `train.csv`:

In [107]:
small_train = pd.read_csv(PATH + '/small_train.csv')

Note that we converted the `date` column to the `datetime` type using `parse_dates`.

In [108]:
print(small_train)

   id        date  store_nbr  item_nbr  unit_sales  onpromotion
0   0  2013-01-01         25    103665         7.0          NaN


In [47]:
types_dict = small_train.dtypes.to_dict()

In [48]:
types_dict

{'date': dtype('O'),
 'id': dtype('int64'),
 'item_nbr': dtype('int64'),
 'onpromotion': dtype('float64'),
 'store_nbr': dtype('int64'),
 'unit_sales': dtype('float64')}

Now, we can use `types_dict` to specify the dtypes of each column of the DataFrame we are loading the `train.csv` file into:

In [22]:
grocery_train = pd.read_csv(PATH + '/train.csv', low_memory=True, dtype=types_dict)

In [30]:
grocery_train.shape  # (No. of observations, No. of features)

(125497040, 6)

Next, we convert the `date` column to dtype `datetime` and add columns showing the `year`, `month`, and `day`.

In [50]:
grocery_train['date'] =  pd.to_datetime(grocery_train['date'])
grocery_train['year'] = grocery_train['date'].dt.year
grocery_train['month'] = grocery_train['date'].dt.month
grocery_train['day'] = grocery_train['date'].dt.day

In [71]:
grocery_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,year,month,day
0,0,2013-01-01,25,103665,7.0,NaN,2013,1,1
1,1,2013-01-01,25,105574,1.0,NaN,2013,1,1
2,2,2013-01-01,25,105575,2.0,NaN,2013,1,1
3,3,2013-01-01,25,108079,1.0,NaN,2013,1,1
4,4,2013-01-01,25,108701,1.0,NaN,2013,1,1


### Test Data
Next, we read in the `test.csv` file:

In [59]:
grocery_test = pd.read_csv(PATH + '/test.csv', low_memory=False, parse_dates=['date'])

In [70]:
grocery_test.head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False
3,125497043,2017-08-16,1,103520,False
4,125497044,2017-08-16,1,103665,False


In [76]:
grocery_test.shape # (No. of observations, No. of features)

(3370464, 5)

### Stores Data

In [81]:
stores = pd.read_csv(PATH + '/stores.csv')
stores.shape # (No. of observations, No. of features)

(54, 5)

In [78]:
stores

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


### Items Data

In [82]:
items = pd.read_csv(PATH + '/items.csv')
items.shape # (No. of observations, No. of features)

(4100, 4)

In [83]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


### Transactions Data

In [84]:
transactions = pd.read_csv(PATH + '/transactions.csv')
transactions.shape # (No. of observations, No. of features)

(83488, 3)

In [85]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### Oil Data

In [115]:
oil = pd.read_csv(PATH + '/oil.csv')
oil['date'] =  pd.to_datetime(oil['date'])
oil.shape # (No. of observations, No. of features)

(1218, 2)

In [116]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


### Holidays Data

In [91]:
holidays = pd.read_csv(PATH + '/holidays_events.csv')
holidays.shape # (No. of observations, No. of features)

(350, 6)

In [92]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


### Creating a Single DataFrame for the Training Data

In [ ]:
grocery_train = pd.merge(grocery_train, stores, how='left', on='store_nbr')
grocery_train = pd.merge(grocery_train, transactions, how='left', on=['store_nbr', 'date'])
grocery_train = pd.merge(grocery_train, items, how='left', on='item_nbr')
grocery_train = pd.merge(grocery_train, oil, how='left', on='date')

# ------------------------ Notes ----------------
# The first 2 joins took an endless amount of time on my local m/c.  May need to drop the add-on day, month, year
# columns - although that may likely not help much.

# After this, need to join grocery_train and holidays
## - need to pay attention to locale - may have to do the join on date + locale (match to city from stores)
## - locale seems to include a province sometimes and a city other times - because it shows where the holiday was in effect

# Can start applying random forest after the above steps are completed

In [ ]:
grocery_train.head()

In [101]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [106]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [109]:
small_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
